In [3]:
from rumour_milled.ml.load import load_headlines
from rumour_milled.ml.preprocess import tokenise_and_vectorise
from rumour_milled.ml.models.simple import SimpleHeadlineClassifier
from rumour_milled.ml.train import Trainer
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
# GeForce Game Ready Driver 577.0
# GTX 1660
# Turing 7.5
# CUDA SDK 10.0-10.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
headlines, labels = load_headlines()
headlines = headlines[:640]
labels = labels[:640]
X_train = tokenise_and_vectorise(headlines[:512], batch_size=128)
y_train = torch.tensor(labels[:512], dtype=torch.float).unsqueeze(1)
X_test = tokenise_and_vectorise(headlines[512:], batch_size=128)
y_test = torch.tensor(labels[512:], dtype=torch.float).unsqueeze(1)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

Vectorising 128/512
Vectorising 256/512
Vectorising 384/512
Vectorising 512/512
Vectorising 128/128


In [6]:
model = SimpleHeadlineClassifier(768, 256, 1)
loss_fn = nn.BCEWithLogitsLoss()  # Binary Cross Entropy for [0,1] output
optimiser = optim.Adam(model.parameters(), lr=0.001)
trainer = Trainer(model, loss_fn, optimiser, device)

In [7]:
trainer.train(
    train_loader,
    test_loader,
    100
)

100%|██████████| 16/16 [00:00<00:00, 426.01it/s]


In [33]:
test_X = tokenise_and_vectorise(["Someone Unveils Self-Driving Bullet Train Connecting Beijing to London"]).to(device)
with torch.no_grad():
    print(trainer.model(test_X))

Vectorising 1/1
tensor([[1.7371e-13]], device='cuda:0')
